## Extract

We will be extracting the file from S3 bucket for transformation and then reloading it to the S3 bucket.

In [1]:
import boto3
import pandas as pd
from io import StringIO

In [2]:
# create s3 client
s3 = boto3.client('s3')

#Identify the 'name "the s3 bucket and
#indicate the name and the "file"

bucket_name = "dseast-weight-dh"
file_name = "diabetes_012_health_indicators_BRFSS2015.csv"

# using these parameters, we get our " diabetes_012_health_indicators_BRFSS2015.csv" object from s3
diabetesfactors_object = s3.get_object(Bucket=bucket_name, Key=file_name)
#diabetesfactors_object = s3.get_object(Bucket=dseast-weight-dh, Key=diabetes_012_health_indicators_BRFSS2015.csv)



In [3]:
# then we subsequently save it into a pandas dataframe to prepare it for transformations
df = pd.read_csv(diabetesfactors_object['Body'])

In [8]:
# Confirm the first 5 rows of the file
df.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


## Transform

We are transforming the table by removing or deleting columns that are not necessarily well suited for the analysis that is currently needed. 
The dropped columns though representatitve of the data model as a whole, they are less impactful in our final analysis

In [4]:

# The following columns will be dropped: 
#Cholestol Check
#GenHlth
#NoDocbcCost

primary_select = ['Diabetes_012', 'HighBP', 'HighChol', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies','HvyAlcoholConsump', 'AnyHealthcare', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age','Education']

diabfactor_df = df[primary_select]

diabfactor_df.head()

,Diabetes_012,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education
0,0.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,18.0,15.0,1.0,0.0,9.0,4.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.0
2,0.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,30.0,30.0,1.0,0.0,9.0,4.0
3,0.0,1.0,0.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,3.0
4,0.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,11.0,5.0


In [12]:
#Check for null values in the dataframe

diabfactor_df.isna().sum()

Diabetes_012            0
HighBP                  0
HighChol                0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
dtype: int64

In [13]:
diabfactor_df.shape

(253680, 18)

In [5]:
# drop all missing values

dropped_df = diabfactor_df.dropna()

In [15]:
dropped_df.shape

(253680, 18)

In [16]:
dropped_df.head()

,Diabetes_012,HighBP,HighChol,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education
0,0.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,18.0,15.0,1.0,0.0,9.0,4.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,6.0
2,0.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,30.0,30.0,1.0,0.0,9.0,4.0
3,0.0,1.0,0.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,11.0,3.0
4,0.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,11.0,5.0


## Load

Now that we have our data transformed and ready for analysis, we will import it back into our S3 bucket. Preferably, we should have a seperate folder for this transformed data (for example, perhaps something named `transformed`) for easier location of our newly transformed dataset.

We will then load this transformed dataset into AWS Athena.

In [6]:
import pandas as pd
from io import StringIO

# create a buffer object (temp place to hold data)
csv_buffer = StringIO()

# open the file in binary format, and save into the var 'data'
dropped_df.to_csv(csv_buffer, index=False)

### ADDING OBJECTS TO A BUCKET ###
response = s3.put_object(
    Body=csv_buffer.getvalue(),
    Bucket= "dseast-weight-dh",
    Key="diabetest-factor-transformed-data.csv"
)